#### Final Project: H&M Personalized Fashion Recommendations
#### Market Basket Analysis (MBA) Preprocessing
#### Will Jarrard (wej5ar) Abhi Dommalapati (ad4bu), Sebastian Ranasinghe (sar2jf)

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%cd /project/ds5559/h_and_m/
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import collect_set, col, count, row_number, lit
from pyspark.sql.window import Window
from pyspark.ml.fpm import FPGrowth
from pyspark.sql.functions import from_unixtime, unix_timestamp, year, month, col, date_format
# spark = SparkSession.builder.getOrCreate()
spark = SparkSession.builder \
    .master('local[*]') \
    .config("spark.driver.memory", "100g") \
    .appName('my-cool-app') \
    .getOrCreate()

/project/ds5559/h_and_m


#### Load Data

In [2]:
customers = spark.read.csv('/project/ds5559/h_and_m/customers.csv',  inferSchema=True, header = True)
articles = spark.read.csv('/project/ds5559/h_and_m/articles.csv',  inferSchema=True, header = True)
transactions_full = spark.read.csv('/project/ds5559/h_and_m/transactions_train.csv',  inferSchema=True, header = True)

In [3]:
transactions_full.show(5)

+----------+--------------------+----------+--------------------+----------------+
|     t_dat|         customer_id|article_id|               price|sales_channel_id|
+----------+--------------------+----------+--------------------+----------------+
|2018-09-20|000058a12d5b43e67...| 663713001|0.050830508474576264|               2|
|2018-09-20|000058a12d5b43e67...| 541518023| 0.03049152542372881|               2|
|2018-09-20|00007d2de826758b6...| 505221004| 0.01523728813559322|               2|
|2018-09-20|00007d2de826758b6...| 685687003|0.016932203389830508|               2|
|2018-09-20|00007d2de826758b6...| 685687004|0.016932203389830508|               2|
+----------+--------------------+----------+--------------------+----------------+
only showing top 5 rows



#### Create Lookup Table for Article Types

In [4]:
# Create distinct pairings of product types, and index

art_lookup = articles.select(col('product_type_name'), col('index_name')) \
                       .distinct() \
                       .sort(col('product_type_no'))


# Create ID for unique product, index types

w = Window().orderBy(lit('A'))
art_lookup = art_lookup.withColumn("prod_index_id", row_number().over(w))


# Rejoin with articles, extract relevant columns

art_lookup = articles.join(art_lookup, 
                           on = ['product_type_name', 'index_name'], 
                           how = 'inner')

art_lookup = art_lookup.select(col('article_id'),
                  col('product_type_name'), col('product_code'),
                  col('product_type_no'), 
                  col('index_name'), 
                  col('prod_index_id')).sort(col('prod_index_id'))

art_lookup_short = art_lookup.select(col('article_id'), 
                  col('prod_index_id')).sort(col('prod_index_id'))

In [5]:
# Full Lookup Table

art_lookup_short.show(5)

+----------+-------------+
|article_id|prod_index_id|
+----------+-------------+
| 519243001|            1|
| 919896001|            2|
| 894221001|            2|
| 902265001|            2|
| 902265002|            2|
+----------+-------------+
only showing top 5 rows



In [6]:
# See that shirts now have a unique ID for each index

art_lookup.select(col('product_type_name'), 
                col('product_type_no'), 
                col('index_name'), 
                col('prod_index_id')).distinct().sort(col('prod_index_id')).filter(col('product_type_name') == 'Shirt').show()

+-----------------+---------------+--------------------+-------------+
|product_type_name|product_type_no|          index_name|prod_index_id|
+-----------------+---------------+--------------------+-------------+
|            Shirt|            259|Children Sizes 13...|          261|
|            Shirt|            259|    Baby Sizes 50-98|          262|
|            Shirt|            259|             Divided|          263|
|            Shirt|            259|Children Sizes 92...|          264|
|            Shirt|            259|          Ladieswear|          265|
|            Shirt|            259|            Menswear|          266|
|            Shirt|            259|Children Accessor...|          267|
+-----------------+---------------+--------------------+-------------+



<br>

#### To reduce size of dataset, only orders from 2020 will be considered

In [9]:
# Used https://stackoverflow.com/questions/53285032/how-do-i-convert-timestamp-to-unix-format-with-pyspark
# and https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.from_unixtime.html

transactions_2020 =  transactions_full.withColumn('t_dat', transactions_full['t_dat'].cast('string'))
transactions_2020 = transactions_2020.withColumn('date', from_unixtime(unix_timestamp('t_dat', 'yyyy-MM-dd')))
transactions_2020 = transactions_2020.withColumn('year', year(col('date')))
transactions_2020 = transactions_2020[transactions_2020['year'] == 2020]

transactions_2020 = transactions_2020.sort(col('customer_id'))

transactions_2020.show(5)

+----------+--------------------+----------+--------------------+----------------+-------------------+----+
|     t_dat|         customer_id|article_id|               price|sales_channel_id|               date|year|
+----------+--------------------+----------+--------------------+----------------+-------------------+----+
|2020-03-21|00000dbacae5abe5e...| 887593002| 0.02159322033898305|               2|2020-03-21 00:00:00|2020|
|2020-03-21|00000dbacae5abe5e...| 795440001|0.014389830508474576|               2|2020-03-21 00:00:00|2020|
|2020-03-21|00000dbacae5abe5e...| 841260003|0.011508474576271186|               2|2020-03-21 00:00:00|2020|
|2020-03-21|00000dbacae5abe5e...| 859416011|0.014389830508474576|               2|2020-03-21 00:00:00|2020|
|2020-03-21|00000dbacae5abe5e...| 890498002|0.031762711864406774|               2|2020-03-21 00:00:00|2020|
+----------+--------------------+----------+--------------------+----------------+-------------------+----+
only showing top 5 rows



In [10]:
# Write to parquet file and create temporary view

transactions_2020.write.mode('overwrite').parquet("trans2020_par")

<br>

#### Create Training Set

##### Create training set by subsetting 2020 transaction list by transactions that occurred on / before July 31, 2020. We will treat transactions that occurred on / after August 1, 2020 as the "future" and make predictions later on.  

In [11]:
# Read from parquet file created in above cell

parquetFile = spark.read.parquet("trans2020_par")
parquetFile.createOrReplaceTempView("trans2020_temp")

In [12]:
transaction_train = spark.sql("SELECT * FROM trans2020_temp WHERE t_dat <= '2020-07-31' limit 5000000")
transaction_train = transaction_train.join(art_lookup_short, on = 'article_id', how = 'inner')

transaction_train.sort(col('customer_id')).show(10)

+----------+----------+--------------------+--------------------+----------------+-------------------+----+-------------+
|article_id|     t_dat|         customer_id|               price|sales_channel_id|               date|year|prod_index_id|
+----------+----------+--------------------+--------------------+----------------+-------------------+----+-------------+
| 835417002|2020-03-02|03746d27db5ebfc0e...|0.042355932203389825|               2|2020-03-02 00:00:00|2020|          258|
| 698286007|2020-06-26|03746d27db5ebfc0e...| 0.02240677966101695|               1|2020-06-26 00:00:00|2020|          415|
| 559601019|2020-06-26|03746d27db5ebfc0e...|0.019423728813559322|               1|2020-06-26 00:00:00|2020|           21|
| 543054016|2020-03-02|03746d27db5ebfc0e...|0.042355932203389825|               2|2020-03-02 00:00:00|2020|          215|
| 863409002|2020-03-02|03746d27db5ebfc0e...|0.033881355932203386|               2|2020-03-02 00:00:00|2020|          258|
| 673638007|2020-03-02|0

In [13]:
#Create temporary dataframe and then extract columns

transaction_train.createOrReplaceTempView('train_temp')
train_temp = spark.sql("select s.prod_index_id, s.customer_id \
                    from train_temp s")

In [14]:
# Write to parquet file

train_temp.write.mode('overwrite').parquet("train_par")

<br>

#### Create Test Set

##### Next the test set is created. As mentioned earlier, we are treating transactions that occurred on / after August 1, 2020 as the "future". 
##### We create a list of unique customers for which we want to predict on. A table is then created which shows by customer ID, the purchases made on / before July 31, 2020 (purchase history), the purchases made on / after August 1, 2020 (future purchases, aka labels). 

In [11]:
test_temp = spark.sql("SELECT * FROM trans2020_temp WHERE t_dat > '2020-07-31' limit 500000")

In [12]:
# Create a list of customers that made purchases on or after August 1, 2020

customer_test = test_temp.select(col('customer_id')).distinct()

In [13]:
# Join the customer list onto the transactions_2020 dataset
# Purpose: Filter to only transactions with these customers

temp = spark.sql("SELECT * FROM trans2020_temp")

transaction_test = temp.join(customer_test, on = 'customer_id', how = 'leftsemi')

In [14]:
# Join art_lookup_short to add the prod_idex_id

transaction_test = transaction_test.join(art_lookup_short, on = 'article_id', how = 'inner')

In [15]:
transaction_test.createOrReplaceTempView("transaction_test_temp")

In [17]:
# Create baskets of items for before August 1, 2020

transaction_test_prior = spark.sql("SELECT prod_index_id, customer_id FROM transaction_test_temp WHERE t_dat <= '2020-07-31'")

# Write to parquet file

transaction_test_prior.write.mode('overwrite').parquet("testprior_par")

In [18]:
# Create baskets of items for on / after August 1, 2020

transaction_test_after = spark.sql("SELECT article_id, customer_id FROM transaction_test_temp WHERE t_dat > '2020-07-31'")

# Write to parquet file

transaction_test_after.write.mode('overwrite').parquet("testafter_par")

In [23]:
test_baskets_after.show(5)

+--------------------+--------------------+
|         customer_id|              labels|
+--------------------+--------------------+
|04daaa60957b280cb...|[794575001, 62448...|
|054324bf3c4451750...|         [806778001]|
|058306b9a1720d1a0...|[924250001, 89616...|
|05a45a5e4c53a72f1...|         [832481003]|
|05a8260d130f082b0...|[716672001, 27038...|
+--------------------+--------------------+
only showing top 5 rows

